In [1]:
import json
import os
import pandas as pd
import numpy as np
from collections import Counter

# File Path Configuration
files = {
    "Option 1 (Baseline) - Case 1": r"C:\Users\bmwlab\Desktop\Testing_Row_Data\option_1\json\processed\op1_100_case_1.json",
    "Option 1 (Baseline) - Case 2": r"C:\Users\bmwlab\Desktop\Testing_Row_Data\option_1\json\processed\op1_100_case_2.json",
    "Option 2 (Context) - Case 1": r"C:\Users\bmwlab\Desktop\Testing_Row_Data\option_2\json\processed\op2_100_case_1.json",
    "Option 2 (Context) - Case 2": r"C:\Users\bmwlab\Desktop\Testing_Row_Data\option_2\json\processed\op2_100_case_2.json"
}

In [2]:
def calculate_hhi(items_list):
    """
    Calculate the Herfindahl-Hirschman Index (HHI).
    Used to measure data concentration. Range from 1/N to 1.0.
    """
    if not items_list:
        return 0.0
    
    total = len(items_list)
    counts = Counter(items_list)
    hhi = sum((count / total) ** 2 for count in counts.values())
    return hhi

def analyze_file(name, filepath):
    """Read JSON and calculate various metrics"""
    if not os.path.exists(filepath):
        print(f"⚠️ Warning: File not found: {filepath}")
        return None

    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            data = json.load(f)
    except Exception as e:
        print(f"❌ Error reading {filepath}: {e}")
        return None

    modules = []
    keys = []
    hierarchy_depths = []
    key_lengths = []

    for item in data:
        # 1. Module Extraction
        mod = item.get('affected_module', 'Unknown')
        modules.append(mod)

        # 2. Configuration Key Extraction and Depth Analysis
        key = item.get('modified_key', '')
        keys.append(key)
        
        if key:
            # Hierarchy depth: Calculate the number of segments separated by dots (e.g., 'a.b.c' -> 3)
            h_depth = len(key.split('.'))
            hierarchy_depths.append(h_depth)
            key_lengths.append(len(key))
        else:
            hierarchy_depths.append(0)
            key_lengths.append(0)

    # Data Calculation
    module_hhi = calculate_hhi(modules)
    key_hhi = calculate_hhi(keys)
    avg_h_depth = np.mean(hierarchy_depths) if hierarchy_depths else 0
    avg_k_len = np.mean(key_lengths) if key_lengths else 0
    
    # Count the most frequent modules
    top_mod_count = Counter(modules).most_common(1)
    top_module_str = f"{top_mod_count[0][0]} ({top_mod_count[0][1]})" if top_mod_count else "None"

    return {
        "Name": name,
        "Total Cases": len(data),
        "Module HHI": module_hhi,
        "Key HHI": key_hhi,
        "Avg Hierarchy Level": avg_h_depth,
        "Avg Key Length": avg_k_len,
        "Top Module": top_module_str
    }

In [3]:
results = []
for name, path in files.items():
    res = analyze_file(name, path)
    if res:
        results.append(res)

if results:
    df = pd.DataFrame(results)
    
    # Value Formatting
    df["Module HHI"] = df["Module HHI"].map("{:.4f}".format)
    df["Key HHI"] = df["Key HHI"].map("{:.4f}".format)
    df["Avg Hierarchy Level"] = df["Avg Hierarchy Level"].map("{:.2f}".format)
    df["Avg Key Length"] = df["Avg Key Length"].map("{:.2f}".format)
    
    # Reorder columns for readability
    cols = ["Name", "Total Cases", "Module HHI", "Key HHI", "Avg Hierarchy Level", "Avg Key Length", "Top Module"]
    df = df[cols]
    
    print("### Fault Concentration & Modification Depth Analysis Results ###")
    display(df) # Display nice table in Notebook
    
    print("\n💡 Metric Interpretation Guide:")
    print("- Higher HHI: Indicates a higher concentration of faults/modifications.")
    print("- Module HHI: Measures if faults are concentrated in specific software modules (e.g., PHY).")
    print("- Key HHI: Measures if specific configuration parameters are repeatedly modified.")
    print("- Avg Hierarchy Level: Higher values indicate that the modified parameters are deeper within the configuration tree.")
else:
    print("No analyzable data found.")

### Fault Concentration & Modification Depth Analysis Results ###


,Name,Total Cases,Module HHI,Key HHI,Avg Hierarchy Level,Avg Key Length,Top Module
0,Option 1 (Baseline) - Case 1,100,0.1998,0.0118,2.46,34.46,RRC (36)
1,Option 1 (Baseline) - Case 2,100,0.2316,0.0132,2.53,35.26,PHY (39)
2,Option 2 (Context) - Case 1,100,0.3034,0.0160,2.78,48.61,PHY (40)
3,Option 2 (Context) - Case 2,100,0.4038,0.0118,2.62,39.56,PHY (60)



💡 Metric Interpretation Guide:
- Higher HHI: Indicates a higher concentration of faults/modifications.
- Module HHI: Measures if faults are concentrated in specific software modules (e.g., PHY).
- Key HHI: Measures if specific configuration parameters are repeatedly modified.
- Avg Hierarchy Level: Higher values indicate that the modified parameters are deeper within the configuration tree.
